In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import numpy as np
import pandas as pd

In [113]:
from sklearn.preprocessing import StandardScaler

In [36]:
from scipy.special import expit, softmax

In [2]:
data = load_breast_cancer()
X = data['data']
y = data['target']

In [9]:
X.shape

(569, 30)

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [4]:
lda = LinearDiscriminantAnalysis(store_covariance=True)
lda.fit(X_train, y_train)

LinearDiscriminantAnalysis(store_covariance=True)

`priors_`

In [25]:
y_train_counts = np.unique(y_train, return_counts=True)[1]
np.alltrue(lda.priors_ == y_train_counts/y_train_counts.sum())

True

`covariance_`

In [124]:
covariance = (lda.priors_[0]*np.cov(X_train[y_train==0], rowvar=False, ddof=0) + lda.priors_[1]*np.cov(X_train[y_train==1], rowvar=False, ddof=0))
np.alltrue(lda.covariance_==covariance)

True

`means_`

In [131]:
means = np.array([X_train[y_train==0].mean(axis=0),X_train[y_train==1].mean(axis=0)])
np.alltrue(lda.means_== means)

True

`xbar_`

In [140]:
print(np.allclose(lda.xbar_, X_train.mean(axis=0)))
print(np.alltrue(lda.xbar_ == np.dot(lda.priors_, lda.means_)))

True
True


In [141]:
lda.classes_

array([0, 1])

In [204]:
Xc = []
for idx, group in enumerate(lda.classes_):
    Xg = X_train[y_train == group, :]
    Xc.append(Xg - lda.means_[idx])
    
Xc = np.concatenate(Xc, axis=0)

In [205]:
n_samples, n_features = X_train.shape
n_classes = len(lda.classes_)

std = Xc.std(axis=0)
std[std == 0] = 1.0
fac = 1.0 / (n_samples - n_classes)

# 2) Within variance scaling
X_train_ = np.sqrt(fac) * (Xc / std)
# SVD of centered (within)scaled data
U, S, Vt = np.linalg.svd(X_train_, full_matrices=False)

rank = np.sum(S > lda.tol)
# Scaling of within covariance is: V' 1/S
scalings = (Vt[:rank] / std).T / S[:rank]

In [206]:

X_train_0 = np.dot(
    (
        (np.sqrt((n_samples * lda.priors_) * fac))
        * (lda.means_ - lda.xbar_).T
    ).T,
    scalings,
)

_, S, Vt = np.linalg.svd(X_train_0, full_matrices=0)

In [207]:
rank = np.sum(S > lda.tol * S[0])
scalings_ = np.dot(scalings, Vt.T[:, :rank])
coef = np.dot(lda.means_ - lda.xbar_, scalings_)

coef_ = np.dot(coef, scalings_.T)

In [212]:
if n_classes==2:
    coef_ = coef_[1, :] - coef_[0, :]

In [214]:
np.allclose(lda.coef_, coef_)

True

In [190]:
coef = np.dot(lda.means_ - lda.xbar_, lda.scalings_)

In [191]:
coef_ = np.dot(coef, lda.scalings_.T)

In [202]:
coef = np.dot(lda.means_ - lda.xbar_, lda.scalings_)

coef_ = np.dot(coef, lda.scalings_.T)

In [203]:
coef_

array([[-2.39072099e+00,  3.38933447e-02,  2.76217125e-01,
         3.98540860e-03, -4.99039797e+00, -6.06588772e+01,
         1.21988159e+01,  5.96158112e+01, -4.10391128e+00,
         7.05131110e+01,  5.24230970e+00, -1.53202645e-01,
        -1.00020920e-01, -1.50998040e-02,  2.19249339e+02,
        -2.67776314e+01, -5.18305402e+01,  2.20224934e+02,
        -1.44863773e+01, -3.65288458e+01,  2.60063524e+00,
         1.16633638e-01, -1.05652990e-01, -1.16577189e-02,
         1.60551493e+00,  9.24937601e+00,  7.46749661e+00,
        -1.60836330e+01,  1.23839100e+01,  1.54462056e+01],
       [ 1.41269877e+00, -2.00278855e-02, -1.63219210e-01,
        -2.35501417e-03,  2.94887153e+00,  3.58438820e+01,
        -7.20839120e+00, -3.52275248e+01,  2.42503848e+00,
        -4.16668383e+01, -3.09772846e+00,  9.05288357e-02,
         5.91032707e-02,  8.92261143e-03, -1.29556427e+02,
         1.58231458e+01,  3.06271374e+01, -1.30132915e+02,
         8.56013203e+00,  2.15852271e+01, -1.53673901e+

In [ ]:
self.intercept_ = -0.5 * np.sum(coef ** 2, axis=1) + np.log(self.priors_)

In [201]:
np.allclose(lda.scalings_, scalings_)

False

In [189]:
lda.coef_

array([[ 3.80341976e+00, -5.39212301e-02, -4.39436335e-01,
        -6.34042278e-03,  7.93926949e+00,  9.65027592e+01,
        -1.94072071e+01, -9.48433359e+01,  6.52894977e+00,
        -1.12179949e+02, -8.34003815e+00,  2.43731481e-01,
         1.59124190e-01,  2.40224154e-02, -3.48805766e+02,
         4.26007772e+01,  8.24576776e+01, -3.50357849e+02,
         2.30465093e+01,  5.81140729e+01, -4.13737425e+00,
        -1.85553514e-01,  1.68084303e-01,  1.85463710e-02,
        -2.55422830e+00, -1.47149164e+01, -1.18801082e+01,
         2.55875979e+01, -1.97016750e+01, -2.45735089e+01]])

In [177]:
lda.coef_

array([[ 3.80341976e+00, -5.39212301e-02, -4.39436335e-01,
        -6.34042278e-03,  7.93926949e+00,  9.65027592e+01,
        -1.94072071e+01, -9.48433359e+01,  6.52894977e+00,
        -1.12179949e+02, -8.34003815e+00,  2.43731481e-01,
         1.59124190e-01,  2.40224154e-02, -3.48805766e+02,
         4.26007772e+01,  8.24576776e+01, -3.50357849e+02,
         2.30465093e+01,  5.81140729e+01, -4.13737425e+00,
        -1.85553514e-01,  1.68084303e-01,  1.85463710e-02,
        -2.55422830e+00, -1.47149164e+01, -1.18801082e+01,
         2.55875979e+01, -1.97016750e+01, -2.45735089e+01]])

In [151]:
Xc.mean(axis=0)

array([-2.13553229e-15,  6.47296624e-15, -2.69069260e-14, -2.97584668e-13,
        3.91323116e-17,  1.93374010e-17, -3.22544189e-18,  9.03581239e-18,
        2.50715200e-17, -4.37684077e-18, -2.02707204e-16, -2.39856974e-16,
       -1.19562480e-16,  1.65220706e-14, -2.14743516e-18, -7.00752033e-18,
        1.70803542e-18, -2.09119962e-18, -6.05056298e-18, -4.34634014e-19,
       -1.02169799e-14, -2.06525883e-15, -1.20245694e-14, -6.47140461e-14,
       -1.76598662e-17,  1.38899881e-16, -1.97949105e-17,  2.33024833e-17,
        6.85654220e-17, -3.81257907e-19])

In [150]:
np.sqrt(fac)

0.046984098573493956

In [144]:
Xc.shape

(455, 30)

In [135]:
lda.xbar_

array([1.41176352e+01, 1.91850330e+01, 9.18822418e+01, 6.54377582e+02,
       9.57440220e-02, 1.03619319e-01, 8.88981451e-02, 4.82798703e-02,
       1.81098681e-01, 6.27567692e-02, 4.02015824e-01, 1.20268681e+00,
       2.85825341e+00, 4.00712989e+01, 6.98907473e-03, 2.56354484e-02,
       3.28236723e-02, 1.18939407e-02, 2.05735121e-02, 3.82045560e-03,
       1.62351033e+01, 2.55356923e+01, 1.07103121e+02, 8.76987033e+02,
       1.31532132e-01, 2.52741802e-01, 2.74594569e-01, 1.14182222e-01,
       2.90502198e-01, 8.38678462e-02])

In [125]:
lda.means_

array([[1.74169231e+01, 2.14923077e+01, 1.15012959e+02, 9.75013609e+02,
        1.02531124e-01, 1.43885148e-01, 1.59455266e-01, 8.67635503e-02,
        1.93533136e-01, 6.26227219e-02, 6.00757988e-01, 1.20041598e+00,
        4.28259763e+00, 7.18094675e+01, 6.75819527e-03, 3.17857870e-02,
        4.18483432e-02, 1.50038935e-02, 2.06236627e-02, 3.97157988e-03,
        2.10274556e+01, 2.92200592e+01, 1.40713964e+02, 1.41022781e+03,
        1.44440769e-01, 3.71363373e-01, 4.51448994e-01, 1.81149467e-01,
        3.26636095e-01, 9.11269822e-02],
       [1.21680559e+01, 1.78216434e+01, 7.82140909e+01, 4.64910839e+02,
        9.17334615e-02, 7.98258741e-02, 4.72053007e-02, 2.55395140e-02,
        1.73751049e-01, 6.28359790e-02, 2.84577273e-01, 1.20402867e+00,
        2.01659545e+00, 2.13169266e+01, 7.12550350e-03, 2.20011573e-02,
        2.74909122e-02, 1.00562413e-02, 2.05438776e-02, 3.73115490e-03,
        1.34032587e+01, 2.33585664e+01, 8.72421678e+01, 5.61890210e+02,
        1.23904301e-01,

In [8]:
lda.coef_.shape

(1, 30)

In [5]:
pred = lda.predict(X_test)
pred_proba = lda.predict_proba(X_test)

In [23]:
np.unique(y_train, return_counts=True)

(array([0, 1]), array([169, 286], dtype=int64))

In [21]:
169/sum([169, 286]) == lda.priors_[0]

True

In [55]:
lda.priors_

array([0.37142857, 0.62857143])

In [114]:
xt = StandardScaler().fit_transform(X_train)
#X_train-X_train.mean(axis=0)[np.newaxis, :]

In [115]:
u,s,vt = np.linalg.svd(xt, full_matrices=False)
S = np.diag(s)

In [116]:
(vt.T.dot(S.dot(S)).dot(vt)/(xt.shape[0]-1))[0]

array([ 1.00220264,  0.30755659,  1.00004841,  0.98878206,  0.17343168,
        0.47903914,  0.64873693,  0.81416624,  0.15071689, -0.33631423,
        0.66842324, -0.09238826,  0.65751645,  0.72106264, -0.20118932,
        0.15991995,  0.14813895,  0.3360734 , -0.09351128, -0.07574188,
        0.97126871,  0.288223  ,  0.96565783,  0.94392388,  0.12219483,
        0.38717613,  0.49789431,  0.7238998 ,  0.16846208, -0.02439076])

In [118]:
(xt.T.dot(xt)/(xt.shape[0]-1))[0]

array([ 1.00220264,  0.30755659,  1.00004841,  0.98878206,  0.17343168,
        0.47903914,  0.64873693,  0.81416624,  0.15071689, -0.33631423,
        0.66842324, -0.09238826,  0.65751645,  0.72106264, -0.20118932,
        0.15991995,  0.14813895,  0.3360734 , -0.09351128, -0.07574188,
        0.97126871,  0.288223  ,  0.96565783,  0.94392388,  0.12219483,
        0.38717613,  0.49789431,  0.7238998 ,  0.16846208, -0.02439076])

In [117]:
np.cov(xt, rowvar=False)[0]

array([ 1.00220264,  0.30755659,  1.00004841,  0.98878206,  0.17343168,
        0.47903914,  0.64873693,  0.81416624,  0.15071689, -0.33631423,
        0.66842324, -0.09238826,  0.65751645,  0.72106264, -0.20118932,
        0.15991995,  0.14813895,  0.3360734 , -0.09351128, -0.07574188,
        0.97126871,  0.288223  ,  0.96565783,  0.94392388,  0.12219483,
        0.38717613,  0.49789431,  0.7238998 ,  0.16846208, -0.02439076])

In [120]:
(lda.priors_[0]*np.cov(X_train[y_train==0], rowvar=False, ddof=0) + lda.priors_[1]*np.cov(X_train[y_train==1], rowvar=False, ddof=0))[0]

array([ 6.04228948e+00,  1.20539129e-01,  4.05295740e+01,  6.10380876e+02,
       -4.73158249e-03,  9.98255704e-03,  4.39274720e-02,  3.40574177e-02,
       -9.65809074e-03, -8.27441332e-03,  2.78826409e-01, -1.71767204e-01,
        2.01724877e+00,  5.80262272e+01, -1.71486953e-03, -1.51562701e-03,
       -8.30639256e-04,  1.38330789e-03, -2.78787171e-03, -1.03777153e-03,
        7.10736302e+00, -1.02913267e+00,  4.79266935e+01,  8.46717039e+02,
       -1.52372115e-02, -1.99819570e-02,  2.22271372e-02,  3.59149565e-02,
       -3.29949785e-02, -1.56847273e-02])

In [68]:
lda.covariance_[0]

array([ 6.04228948e+00,  1.20539129e-01,  4.05295740e+01,  6.10380876e+02,
       -4.73158249e-03,  9.98255704e-03,  4.39274720e-02,  3.40574177e-02,
       -9.65809074e-03, -8.27441332e-03,  2.78826409e-01, -1.71767204e-01,
        2.01724877e+00,  5.80262272e+01, -1.71486953e-03, -1.51562701e-03,
       -8.30639256e-04,  1.38330789e-03, -2.78787171e-03, -1.03777153e-03,
        7.10736302e+00, -1.02913267e+00,  4.79266935e+01,  8.46717039e+02,
       -1.52372115e-02, -1.99819570e-02,  2.22271372e-02,  3.59149565e-02,
       -3.29949785e-02, -1.56847273e-02])

In [48]:
np.linalg.inv(lda.covariance_).dot(lda.means_.T)

array([[ 5.75253064e+01,  6.13455183e+01],
       [-4.04550731e-01, -4.58710024e-01],
       [ 9.42562114e+00,  8.98424469e+00],
       [-9.17354375e-01, -9.23722791e-01],
       [-1.62413462e+02, -1.54439141e+02],
       [-3.07018153e+03, -2.97325271e+03],
       [ 2.95993275e+02,  2.76500385e+02],
       [-4.92171899e+02, -5.87433970e+02],
       [ 5.08396651e+02,  5.14954426e+02],
       [ 1.80087552e+04,  1.78960800e+04],
       [ 3.69151815e+01,  2.85383220e+01],
       [-1.57302603e+01, -1.54854527e+01],
       [-5.86467743e+00, -5.70485071e+00],
       [ 5.81703701e-01,  6.05832176e-01],
       [ 1.91807097e+03,  1.56772522e+03],
       [ 1.66896778e+03,  1.71175664e+03],
       [-3.18468555e+02, -2.35646826e+02],
       [ 1.33457246e+03,  9.82667782e+02],
       [ 7.31894505e+02,  7.55042764e+02],
       [-1.26198141e+04, -1.25614435e+04],
       [-1.46923070e+01, -1.88479478e+01],
       [ 3.03863830e+00,  2.85226557e+00],
       [ 1.24280836e+00,  1.41163475e+00],
       [ 4.

In [123]:
lda.scalings_

array([[-9.81072667e-01],
       [ 1.39087054e-02],
       [ 1.13350354e-01],
       [ 1.63547962e-03],
       [-2.04789394e+00],
       [-2.48923930e+01],
       [ 5.00598977e+00],
       [ 2.44643532e+01],
       [-1.68410919e+00],
       [ 2.89362440e+01],
       [ 2.15127017e+00],
       [-6.28692885e-02],
       [-4.10452708e-02],
       [-6.19645914e-03],
       [ 8.99726628e+01],
       [-1.09886525e+01],
       [-2.12695360e+01],
       [ 9.03730146e+01],
       [-5.94472916e+00],
       [-1.49902278e+01],
       [ 1.06721452e+00],
       [ 4.78625797e-02],
       [-4.33564859e-02],
       [-4.78394151e-03],
       [ 6.58850120e-01],
       [ 3.79563739e+00],
       [ 3.06441314e+00],
       [-6.60018997e+00],
       [ 5.08194627e+00],
       [ 6.33861088e+00]])

In [43]:
lda.coef_

array([[ 3.80341976e+00, -5.39212301e-02, -4.39436335e-01,
        -6.34042278e-03,  7.93926949e+00,  9.65027592e+01,
        -1.94072071e+01, -9.48433359e+01,  6.52894977e+00,
        -1.12179949e+02, -8.34003815e+00,  2.43731481e-01,
         1.59124190e-01,  2.40224154e-02, -3.48805766e+02,
         4.26007772e+01,  8.24576776e+01, -3.50357849e+02,
         2.30465093e+01,  5.81140729e+01, -4.13737425e+00,
        -1.85553514e-01,  1.68084303e-01,  1.85463710e-02,
        -2.55422830e+00, -1.47149164e+01, -1.18801082e+01,
         2.55875979e+01, -1.97016750e+01, -2.45735089e+01]])

In [38]:
expit(lda.decision_function(X_test))== pred_proba[:,1]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [42]:
lda.decision_function(X_test) ==(X_test.dot(lda.coef_.T) + lda.intercept_).flatten()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [97]:
class1 = X_train[y_train==1, :]

In [137]:
class0 = X_train[y_train==0, :]

In [156]:
np.alltrue((lda.priors_[0] * np.cov(class0.T, ddof=0) + lda.priors_[1] * np.cov(class1.T, ddof=0)) == lda.covariance_)

True

In [163]:
np.allclose(lda.xbar_ ,X_train.mean(axis=0))

True

In [165]:
lda.means_.shape

(2, 30)

In [162]:
X_train.mean(axis=0)

array([1.41176352e+01, 1.91850330e+01, 9.18822418e+01, 6.54377582e+02,
       9.57440220e-02, 1.03619319e-01, 8.88981451e-02, 4.82798703e-02,
       1.81098681e-01, 6.27567692e-02, 4.02015824e-01, 1.20268681e+00,
       2.85825341e+00, 4.00712989e+01, 6.98907473e-03, 2.56354484e-02,
       3.28236723e-02, 1.18939407e-02, 2.05735121e-02, 3.82045560e-03,
       1.62351033e+01, 2.55356923e+01, 1.07103121e+02, 8.76987033e+02,
       1.31532132e-01, 2.52741802e-01, 2.74594569e-01, 1.14182222e-01,
       2.90502198e-01, 8.38678462e-02])

In [154]:
lda.covariance_[0]

array([ 6.04228948e+00,  1.20539129e-01,  4.05295740e+01,  6.10380876e+02,
       -4.73158249e-03,  9.98255704e-03,  4.39274720e-02,  3.40574177e-02,
       -9.65809074e-03, -8.27441332e-03,  2.78826409e-01, -1.71767204e-01,
        2.01724877e+00,  5.80262272e+01, -1.71486953e-03, -1.51562701e-03,
       -8.30639256e-04,  1.38330789e-03, -2.78787171e-03, -1.03777153e-03,
        7.10736302e+00, -1.02913267e+00,  4.79266935e+01,  8.46717039e+02,
       -1.52372115e-02, -1.99819570e-02,  2.22271372e-02,  3.59149565e-02,
       -3.29949785e-02, -1.56847273e-02])

In [103]:
lda.priors_

array([0.37142857, 0.62857143])

In [105]:
np.unique(y_train, return_counts=True)

(array([0, 1]), array([169, 286], dtype=int64))

In [108]:
286/sum([169, 286])

0.6285714285714286

In [117]:
X_train.mean(axis=0)

array([1.41176352e+01, 1.91850330e+01, 9.18822418e+01, 6.54377582e+02,
       9.57440220e-02, 1.03619319e-01, 8.88981451e-02, 4.82798703e-02,
       1.81098681e-01, 6.27567692e-02, 4.02015824e-01, 1.20268681e+00,
       2.85825341e+00, 4.00712989e+01, 6.98907473e-03, 2.56354484e-02,
       3.28236723e-02, 1.18939407e-02, 2.05735121e-02, 3.82045560e-03,
       1.62351033e+01, 2.55356923e+01, 1.07103121e+02, 8.76987033e+02,
       1.31532132e-01, 2.52741802e-01, 2.74594569e-01, 1.14182222e-01,
       2.90502198e-01, 8.38678462e-02])

In [126]:
np.cov(X_train,)/X_train.shape[0]

array([[ 11.88091078,  70.28598927,  11.72933594, ...,  26.68844124,
         30.34848587,  20.56758669],
       [ 70.28598927, 426.83773003,  69.08701018, ..., 158.02538386,
        182.38088368, 121.9995804 ],
       [ 11.72933594,  69.08701018,  11.60669297, ...,  26.44940057,
         29.88200846,  20.33577591],
       ...,
       [ 26.68844124, 158.02538386,  26.44940057, ...,  60.91468933,
         68.29570918,  46.60544606],
       [ 30.34848587, 182.38088368,  29.88200846, ...,  68.29570918,
         78.28943866,  52.66223585],
       [ 20.56758669, 121.9995804 ,  20.33577591, ...,  46.60544606,
         52.66223585,  35.78854373]])

In [134]:
(np.linalg.inv(lda.covariance_)).dot(lda.means_[0].T)

array([ 5.75253064e+01, -4.04550731e-01,  9.42562114e+00, -9.17354375e-01,
       -1.62413462e+02, -3.07018153e+03,  2.95993275e+02, -4.92171899e+02,
        5.08396651e+02,  1.80087552e+04,  3.69151815e+01, -1.57302603e+01,
       -5.86467743e+00,  5.81703701e-01,  1.91807097e+03,  1.66896778e+03,
       -3.18468555e+02,  1.33457246e+03,  7.31894505e+02, -1.26198141e+04,
       -1.46923070e+01,  3.03863830e+00,  1.24280836e+00,  4.94055866e-02,
        7.31078127e+02,  1.86560536e+02, -3.94592933e+00, -2.03354141e+02,
       -5.94187619e+01, -1.24631799e+03])

In [136]:
(np.linalg.inv(lda.covariance_)).dot(lda.means_.mean(axis=0).T)

array([ 5.94354124e+01, -4.31630378e-01,  9.20493291e+00, -9.20538583e-01,
       -1.58426302e+02, -3.02171712e+03,  2.86246830e+02, -5.39802934e+02,
        5.11675538e+02,  1.79524176e+04,  3.27267518e+01, -1.56078565e+01,
       -5.78476407e+00,  5.93767939e-01,  1.74289809e+03,  1.69036221e+03,
       -2.77057691e+02,  1.15862012e+03,  7.43468635e+02, -1.25906288e+04,
       -1.67701274e+01,  2.94545194e+00,  1.32722155e+00,  5.87197133e-02,
        7.29795374e+02,  1.79170595e+02, -9.91220886e+00, -1.90503857e+02,
       -6.93130910e+01, -1.25865899e+03])

In [127]:
lda.coef_

array([[ 3.80341976e+00, -5.39212301e-02, -4.39436335e-01,
        -6.34042278e-03,  7.93926949e+00,  9.65027592e+01,
        -1.94072071e+01, -9.48433359e+01,  6.52894977e+00,
        -1.12179949e+02, -8.34003815e+00,  2.43731481e-01,
         1.59124190e-01,  2.40224154e-02, -3.48805766e+02,
         4.26007772e+01,  8.24576776e+01, -3.50357849e+02,
         2.30465093e+01,  5.81140729e+01, -4.13737425e+00,
        -1.85553514e-01,  1.68084303e-01,  1.85463710e-02,
        -2.55422830e+00, -1.47149164e+01, -1.18801082e+01,
         2.55875979e+01, -1.97016750e+01, -2.45735089e+01]])

In [100]:
class1.mean(axis=0) == lda.means_[1]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])